# Imports

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import scipy.stats as st
#from pandas_profiling import ProfileReport
#import missingno as msno

In [2]:
ls -lh ./data

total 353M
-rw-r--r-- 1 mijka mijka 137K 22 janv.  2018 EdStatsCountry.csv
-rw-r--r-- 1 mijka mijka  48K 22 janv.  2018 EdStatsCountry-Series.csv
-rw-r--r-- 1 mijka mijka 312M 22 janv.  2018 EdStatsData.csv
-rw-r--r-- 1 mijka mijka  38M 22 janv.  2018 EdStatsFootNote.csv
-rw-r--r-- 1 mijka mijka 3,6M 22 janv.  2018 EdStatsSeries.csv


# Situation initale, objectifs


## Situation initiale

### Contexte


Vous êtes Data Scientist dans une start-up de la EdTech, nommée academy, qui propose des contenus de formation en ligne pour un public de **niveau lycée et université**.

Mark, votre manager, vous a convié à une réunion pour vous présenter le **projet d’expansion à l’international de l’entreprise**. Il vous confie une première mission d’analyse exploratoire, pour **déterminer si les données sur l’éducation de la banque mondiale permettent d’informer le projet d’expansion**.

### Questions lors de la réunion

Voici les différentes questions que Mark aimerait explorer, que vous avez notées durant la réunion

- Quels sont les pays avec un fort potentiel de clients pour nos services ?
- Pour chacun de ces pays, quelle sera l’évolution de ce potentiel de clients ?
- Dans quels pays l'entreprise doit-elle opérer en priorité ?

### Mail post-réunion


#### Description Source Data

"Les données de la Banque mondiale sont disponibles à l’adresse suivante : https://datacatalog.worldbank.org/dataset/education-statistics

Ou en téléchargement direct à ce [lien](https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/Parcours_data_scientist/Projet+-+Donn%C3%A9es+%C3%A9ducatives/Projet+Python_Dataset_Edstats_csv.zip)."

> The World Bank EdStats All Indicator Query holds over 4,000 internationally comparable indicators that describe education access, progression, completion, literacy, teachers, population, and expenditures. The indicators cover the education cycle from pre-primary to vocational and tertiary education.The query also holds learning outcome data from international and regional learning assessments (e.g. PISA, TIMSS, PIRLS), equity data from household surveys, and projection/attainment data to 2050. For further information, please visit the EdStats website.

"Je te laisse regarder la page d'accueil qui décrit le jeu de données. En résumé, l’organisme “EdStats All Indicator Query” de la Banque mondiale répertorie 4000 indicateurs internationaux décrivant l’accès à l’éducation, l’obtention de diplômes et des informations relatives aux professeurs, aux dépenses liées à l’éducation... Tu trouveras plus d'info [sur ce site](http://datatopics.worldbank.org/education/)"

#### Requêtes supplémentaires

Pour la pré-analyse, pourrais-tu :

- Valider la qualité de ce jeu de données (comporte-t-il beaucoup de données manquantes, dupliquées ?)
- Décrire les informations contenues dans le jeu de données (nombre de colonnes ? nombre de lignes ?)
- Sélectionner les informations qui semblent pertinentes pour répondre à la problématique (quelles sont les colonnes contenant des informations qui peuvent être utiles pour répondre à la problématique de l’entreprise ?)
- Déterminer des ordres de grandeurs des indicateurs statistiques classiques pour les différentes zones géographiques et pays du monde (moyenne/médiane/écart-type par pays et par continent ou bloc géographique)

Ton travail va nous permettre de déterminer si ce jeu de données peut informer les décisions d'ouverture vers de nouveaux pays. On va partager ton analyse avec le board, alors merci de soigner la présentation et de l'illustrer avec des graphiques pertinents et lisibles !

###  Objectif :

"Permettre de déterminer si ce jeu de données peut informer les décisions d'ouverture vers de nouveaux pays. On va partager ton analyse avec le board, alors merci de soigner la présentation et de l'illustrer avec des graphiques pertinents et lisibles"

# Exploration

## Main DFs

### data (ED_Stats)

In [3]:
data  = pd.read_csv('./data/EdStatsData.csv')
data.shape

(886930, 70)

In [4]:
data.dtypes

Country Name       object
Country Code       object
Indicator Name     object
Indicator Code     object
1970              float64
                   ...   
2085              float64
2090              float64
2095              float64
2100              float64
Unnamed: 69       float64
Length: 70, dtype: object

In [5]:
data.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1970,1971,1972,1973,1974,1975,...,2060,2065,2070,2075,2080,2085,2090,2095,2100,Unnamed: 69
0,Arab World,ARB,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arab World,ARB,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2.F,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arab World,ARB,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2.GPI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arab World,ARB,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2.M,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arab World,ARB,"Adjusted net enrolment rate, primary, both sex...",SE.PRM.TENR,54.822121,54.894138,56.209438,57.267109,57.991138,59.36554,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# recherche de valeurs manquantes
data_na = data.isnull().sum()
data_na

Country Name           0
Country Code           0
Indicator Name         0
Indicator Code         0
1970              814642
                   ...  
2085              835494
2090              835494
2095              835494
2100              835494
Unnamed: 69       886930
Length: 70, dtype: int64

In [7]:
data.isnull().mean()

Country Name      0.000000
Country Code      0.000000
Indicator Name    0.000000
Indicator Code    0.000000
1970              0.918496
                    ...   
2085              0.942007
2090              0.942007
2095              0.942007
2100              0.942007
Unnamed: 69       1.000000
Length: 70, dtype: float64

In [8]:
# variables ayant des valeurs manquantes
data_na[data_na>0]

1970           814642
1971           851393
1972           851311
1973           851385
1974           851200
                ...  
2085           835494
2090           835494
2095           835494
2100           835494
Unnamed: 69    886930
Length: 66, dtype: int64

In [9]:
# recherche de duplicats
data.duplicated().sum()

0

In [10]:
data.duplicated(subset=['Country Name']).sum()

886688

In [11]:
data.duplicated(subset=['Country Code']).sum()

886688

In [12]:
data.duplicated(subset=['Indicator Name']).sum()

883265

In [13]:
data

,Country Name,Country Code,Indicator Name,Indicator Code,1970,1971,1972,1973,1974,1975,...,2060,2065,2070,2075,2080,2085,2090,2095,2100,Unnamed: 69
0,Arab World,ARB,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arab World,ARB,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2.F,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arab World,ARB,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2.GPI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arab World,ARB,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2.M,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arab World,ARB,"Adjusted net enrolment rate, primary, both sex...",SE.PRM.TENR,54.822121,54.894138,56.209438,57.267109,57.991138,59.36554,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886925,Zimbabwe,ZWE,"Youth illiterate population, 15-24 years, male...",UIS.LP.AG15T24.M,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
886926,Zimbabwe,ZWE,"Youth literacy rate, population 15-24 years, b...",SE.ADT.1524.LT.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
886927,Zimbabwe,ZWE,"Youth literacy rate, population 15-24 years, f...",SE.ADT.1524.LT.FE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
886928,Zimbabwe,ZWE,"Youth literacy rate, population 15-24 years, g...",SE.ADT.1524.LT.FM.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
data_na.dropna()

Country Name           0
Country Code           0
Indicator Name         0
Indicator Code         0
1970              814642
                   ...  
2085              835494
2090              835494
2095              835494
2100              835494
Unnamed: 69       886930
Length: 70, dtype: int64

In [15]:
data.shape[0]

886930

In [16]:
#sns.heatmap(data.isnull(), cbar=False)

In [17]:
#msno.matrix(data)

In [18]:
# détail
data.loc[data['1970'].isnull()]

,Country Name,Country Code,Indicator Name,Indicator Code,1970,1971,1972,1973,1974,1975,...,2060,2065,2070,2075,2080,2085,2090,2095,2100,Unnamed: 69
0,Arab World,ARB,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arab World,ARB,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2.F,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arab World,ARB,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2.GPI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arab World,ARB,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2.M,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Arab World,ARB,"Adjusted net enrolment rate, upper secondary, ...",UIS.NERA.3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886925,Zimbabwe,ZWE,"Youth illiterate population, 15-24 years, male...",UIS.LP.AG15T24.M,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
886926,Zimbabwe,ZWE,"Youth literacy rate, population 15-24 years, b...",SE.ADT.1524.LT.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
886927,Zimbabwe,ZWE,"Youth literacy rate, population 15-24 years, f...",SE.ADT.1524.LT.FE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
886928,Zimbabwe,ZWE,"Youth literacy rate, population 15-24 years, g...",SE.ADT.1524.LT.FM.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
#profile = ProfileReport(EDS_stats_df, minimal=True)
#profile.to_file(output_file="./PP_reports/EDS_stats_df.html")

### country

In [20]:
country  = pd.read_csv('./data/EdStatsCountry.csv')
country.shape

(241, 32)

In [21]:
country.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 32 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   Country Code                                       241 non-null    object 
 1   Short Name                                         241 non-null    object 
 2   Table Name                                         241 non-null    object 
 3   Long Name                                          241 non-null    object 
 4   2-alpha code                                       238 non-null    object 
 5   Currency Unit                                      215 non-null    object 
 6   Special Notes                                      145 non-null    object 
 7   Region                                             214 non-null    object 
 8   Income Group                                       214 non-null    object 
 9   WB-2 code 

In [22]:
country.head()

,Country Code,Short Name,Table Name,Long Name,2-alpha code,Currency Unit,Special Notes,Region,Income Group,WB-2 code,...,IMF data dissemination standard,Latest population census,Latest household survey,Source of most recent Income and expenditure data,Vital registration complete,Latest agricultural census,Latest industrial data,Latest trade data,Latest water withdrawal data,Unnamed: 31
0,ABW,Aruba,Aruba,Aruba,AW,Aruban florin,SNA data for 2000-2011 are updated from offici...,Latin America & Caribbean,High income: nonOECD,AW,...,NaN,2010,NaN,NaN,Yes,NaN,NaN,2012.0,NaN,NaN
1,AFG,Afghanistan,Afghanistan,Islamic State of Afghanistan,AF,Afghan afghani,Fiscal year end: March 20; reporting period fo...,South Asia,Low income,AF,...,General Data Dissemination System (GDDS),1979,"Multiple Indicator Cluster Survey (MICS), 2010/11","Integrated household survey (IHS), 2008",NaN,2013/14,NaN,2012.0,2000,NaN
2,AGO,Angola,Angola,People's Republic of Angola,AO,Angolan kwanza,"April 2013 database update: Based on IMF data,...",Sub-Saharan Africa,Upper middle income,AO,...,General Data Dissemination System (GDDS),1970,"Malaria Indicator Survey (MIS), 2011","Integrated household survey (IHS), 2008",NaN,2015,NaN,NaN,2005,NaN
3,ALB,Albania,Albania,Republic of Albania,AL,Albanian lek,NaN,Europe & Central Asia,Upper middle income,AL,...,General Data Dissemination System (GDDS),2011,"Demographic and Health Survey (DHS), 2008/09",Living Standards Measurement Study Survey (LSM...,Yes,2012,2010.0,2012.0,2006,NaN
4,AND,Andorra,Andorra,Principality of Andorra,AD,Euro,NaN,Europe & Central Asia,High income: nonOECD,AD,...,NaN,2011. Population figures compiled from adminis...,NaN,NaN,Yes,NaN,NaN,2006.0,NaN,NaN


In [23]:
country.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
236    False
237    False
238    False
239    False
240    False
Length: 241, dtype: bool

In [24]:
# recherche de valeurs manquantes
country_na = country.isnull().sum()
country_na

Country Code                                           0
Short Name                                             0
Table Name                                             0
Long Name                                              0
2-alpha code                                           3
Currency Unit                                         26
Special Notes                                         96
Region                                                27
Income Group                                          27
WB-2 code                                              1
National accounts base year                           36
National accounts reference year                     209
SNA price valuation                                   44
Lending category                                      97
Other groups                                         183
System of National Accounts                           26
Alternative conversion factor                        194
PPP survey year                

In [25]:
country.isnull().mean()

Country Code                                         0.000000
Short Name                                           0.000000
Table Name                                           0.000000
Long Name                                            0.000000
2-alpha code                                         0.012448
Currency Unit                                        0.107884
Special Notes                                        0.398340
Region                                               0.112033
Income Group                                         0.112033
WB-2 code                                            0.004149
National accounts base year                          0.149378
National accounts reference year                     0.867220
SNA price valuation                                  0.182573
Lending category                                     0.402490
Other groups                                         0.759336
System of National Accounts                          0.107884
Alternat

In [26]:
# variables ayant des valeurs manquantes
country_na[country_na>0]

2-alpha code                                           3
Currency Unit                                         26
Special Notes                                         96
Region                                                27
Income Group                                          27
WB-2 code                                              1
National accounts base year                           36
National accounts reference year                     209
SNA price valuation                                   44
Lending category                                      97
Other groups                                         183
System of National Accounts                           26
Alternative conversion factor                        194
PPP survey year                                       96
Balance of Payments Manual in use                     60
External debt Reporting status                       117
System of trade                                       41
Government Accounting concept  

In [27]:
#profile_EDSSC = ProfileReport(EDS_statscountry_df, minimal=True)
#profile_EDSSC.to_file(output_file="./PP_reports/EDS_statscountry_df.html")

## Secondary DFs

### country_series / EdStatsCountry : Description des SeriesCode

In [28]:
country_series = pd.read_csv('./data/EdStatsCountry.csv')
country_series.shape

(241, 32)

In [29]:
country_series.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 32 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   Country Code                                       241 non-null    object 
 1   Short Name                                         241 non-null    object 
 2   Table Name                                         241 non-null    object 
 3   Long Name                                          241 non-null    object 
 4   2-alpha code                                       238 non-null    object 
 5   Currency Unit                                      215 non-null    object 
 6   Special Notes                                      145 non-null    object 
 7   Region                                             214 non-null    object 
 8   Income Group                                       214 non-null    object 
 9   WB-2 code 

In [30]:
country_series.head()

,Country Code,Short Name,Table Name,Long Name,2-alpha code,Currency Unit,Special Notes,Region,Income Group,WB-2 code,...,IMF data dissemination standard,Latest population census,Latest household survey,Source of most recent Income and expenditure data,Vital registration complete,Latest agricultural census,Latest industrial data,Latest trade data,Latest water withdrawal data,Unnamed: 31
0,ABW,Aruba,Aruba,Aruba,AW,Aruban florin,SNA data for 2000-2011 are updated from offici...,Latin America & Caribbean,High income: nonOECD,AW,...,NaN,2010,NaN,NaN,Yes,NaN,NaN,2012.0,NaN,NaN
1,AFG,Afghanistan,Afghanistan,Islamic State of Afghanistan,AF,Afghan afghani,Fiscal year end: March 20; reporting period fo...,South Asia,Low income,AF,...,General Data Dissemination System (GDDS),1979,"Multiple Indicator Cluster Survey (MICS), 2010/11","Integrated household survey (IHS), 2008",NaN,2013/14,NaN,2012.0,2000,NaN
2,AGO,Angola,Angola,People's Republic of Angola,AO,Angolan kwanza,"April 2013 database update: Based on IMF data,...",Sub-Saharan Africa,Upper middle income,AO,...,General Data Dissemination System (GDDS),1970,"Malaria Indicator Survey (MIS), 2011","Integrated household survey (IHS), 2008",NaN,2015,NaN,NaN,2005,NaN
3,ALB,Albania,Albania,Republic of Albania,AL,Albanian lek,NaN,Europe & Central Asia,Upper middle income,AL,...,General Data Dissemination System (GDDS),2011,"Demographic and Health Survey (DHS), 2008/09",Living Standards Measurement Study Survey (LSM...,Yes,2012,2010.0,2012.0,2006,NaN
4,AND,Andorra,Andorra,Principality of Andorra,AD,Euro,NaN,Europe & Central Asia,High income: nonOECD,AD,...,NaN,2011. Population figures compiled from adminis...,NaN,NaN,Yes,NaN,NaN,2006.0,NaN,NaN


In [31]:
#profile_EDSCS = ProfileReport(EDS_countryseries_df, minimal=True)
#profile_EDSCS.to_file(output_file="./PP_reports/EDS_countryseries_df.html")

In [32]:
# recherche de valeurs manquantes
country_series_na = country_series.isnull().sum()
country_series_na

Country Code                                           0
Short Name                                             0
Table Name                                             0
Long Name                                              0
2-alpha code                                           3
Currency Unit                                         26
Special Notes                                         96
Region                                                27
Income Group                                          27
WB-2 code                                              1
National accounts base year                           36
National accounts reference year                     209
SNA price valuation                                   44
Lending category                                      97
Other groups                                         183
System of National Accounts                           26
Alternative conversion factor                        194
PPP survey year                

### foot_note

In [33]:
foot_note = pd.read_csv('./data/EdStatsFootNote.csv')
foot_note.shape

(643638, 5)

In [34]:
foot_note.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 643638 entries, 0 to 643637
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   CountryCode  643638 non-null  object 
 1   SeriesCode   643638 non-null  object 
 2   Year         643638 non-null  object 
 3   DESCRIPTION  643638 non-null  object 
 4   Unnamed: 4   0 non-null       float64
dtypes: float64(1), object(4)
memory usage: 24.6+ MB


In [35]:
foot_note.head()

,CountryCode,SeriesCode,Year,DESCRIPTION,Unnamed: 4
0,ABW,SE.PRE.ENRL.FE,YR2001,Country estimation.,NaN
1,ABW,SE.TER.TCHR.FE,YR2005,Country estimation.,NaN
2,ABW,SE.PRE.TCHR.FE,YR2000,Country estimation.,NaN
3,ABW,SE.SEC.ENRL.GC,YR2004,Country estimation.,NaN
4,ABW,SE.PRE.TCHR,YR2006,Country estimation.,NaN


In [36]:
foot_note.SeriesCode.head(30)

0           SE.PRE.ENRL.FE
1           SE.TER.TCHR.FE
2           SE.PRE.TCHR.FE
3           SE.SEC.ENRL.GC
4              SE.PRE.TCHR
5              SE.PRE.NENR
6        SE.SEC.ENRL.VO.FE
7           SE.SEC.ENRL.GC
8           SE.PRM.TCHR.FE
9        SE.PRE.TCHR.FE.ZS
10             SE.PRE.ENRL
11          SE.PRE.NENR.FE
12    SE.SEC.ENRL.VO.FE.ZS
13          SE.SEC.TCHR.FE
14    SE.SEC.ENRL.VO.FE.ZS
15          SE.SEC.ENRL.VO
16       SE.PRE.TCHR.FE.ZS
17          SE.PRE.ENRL.FE
18          SE.PRE.NENR.MA
19          SE.SEC.TCHR.FE
20             SE.PRE.ENRL
21             SE.PRE.TCHR
22             SE.PRE.NENR
23          SE.PRM.TCHR.FE
24          SE.PRE.TCHR.FE
25             SE.PRE.ENRL
26          SE.SEC.TCHR.FE
27          SE.SEC.ENRL.GC
28          SE.PRM.TCHR.FE
29       SE.SEC.ENRL.VO.FE
Name: SeriesCode, dtype: object

In [37]:
type(foot_note.SeriesCode)

pandas.core.series.Series

In [38]:
#profile_EDSSfoot = ProfileReport(foot_note, minimal=True)
#profile_EDSSfoot.to_file(output_file="./PP_reports/EDS_statsfootnote_df.html")

In [39]:
# recherche de valeurs manquantes
foot_note_na = foot_note.isnull().sum()
foot_note_na

CountryCode         0
SeriesCode          0
Year                0
DESCRIPTION         0
Unnamed: 4     643638
dtype: int64

### series

In [40]:
series = pd.read_csv("./data/EdStatsSeries.csv")
series.shape

(3665, 21)

In [41]:
series.head()

,Series Code,Topic,Indicator Name,Short definition,Long definition,Unit of measure,Periodicity,Base Period,Other notes,Aggregation method,...,Notes from original source,General comments,Source,Statistical concept and methodology,Development relevance,Related source links,Other web links,Related indicators,License Type,Unnamed: 20
0,BAR.NOED.1519.FE.ZS,Attainment,Barro-Lee: Percentage of female population age...,Percentage of female population age 15-19 with...,Percentage of female population age 15-19 with...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Robert J. Barro and Jong-Wha Lee: http://www.b...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BAR.NOED.1519.ZS,Attainment,Barro-Lee: Percentage of population age 15-19 ...,Percentage of population age 15-19 with no edu...,Percentage of population age 15-19 with no edu...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Robert J. Barro and Jong-Wha Lee: http://www.b...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BAR.NOED.15UP.FE.ZS,Attainment,Barro-Lee: Percentage of female population age...,Percentage of female population age 15+ with n...,Percentage of female population age 15+ with n...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Robert J. Barro and Jong-Wha Lee: http://www.b...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BAR.NOED.15UP.ZS,Attainment,Barro-Lee: Percentage of population age 15+ wi...,Percentage of population age 15+ with no educa...,Percentage of population age 15+ with no educa...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Robert J. Barro and Jong-Wha Lee: http://www.b...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BAR.NOED.2024.FE.ZS,Attainment,Barro-Lee: Percentage of female population age...,Percentage of female population age 20-24 with...,Percentage of female population age 20-24 with...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Robert J. Barro and Jong-Wha Lee: http://www.b...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
# recherche de valeurs manquantes
series_na = series.isnull().sum()
series_na

Series Code                               0
Topic                                     0
Indicator Name                            0
Short definition                       1509
Long definition                           0
Unit of measure                        3665
Periodicity                            3566
Base Period                            3351
Other notes                            3113
Aggregation method                     3618
Limitations and exceptions             3651
Notes from original source             3665
General comments                       3651
Source                                    0
Statistical concept and methodology    3642
Development relevance                  3662
Related source links                   3450
Other web links                        3665
Related indicators                     3665
License Type                           3665
Unnamed: 20                            3665
dtype: int64

# Réponses questions :

## Qualité du jeu de données (comporte-t-il beaucoup de données manquantes, dupliquées ?)

- beaucoup de données manquantes
- colonnes mal typées (dates en "object")

### données manquantes

In [43]:
data_na

Country Name           0
Country Code           0
Indicator Name         0
Indicator Code         0
1970              814642
                   ...  
2085              835494
2090              835494
2095              835494
2100              835494
Unnamed: 69       886930
Length: 70, dtype: int64

In [44]:
country_na

Country Code                                           0
Short Name                                             0
Table Name                                             0
Long Name                                              0
2-alpha code                                           3
Currency Unit                                         26
Special Notes                                         96
Region                                                27
Income Group                                          27
WB-2 code                                              1
National accounts base year                           36
National accounts reference year                     209
SNA price valuation                                   44
Lending category                                      97
Other groups                                         183
System of National Accounts                           26
Alternative conversion factor                        194
PPP survey year                

In [45]:
country_series_na

Country Code                                           0
Short Name                                             0
Table Name                                             0
Long Name                                              0
2-alpha code                                           3
Currency Unit                                         26
Special Notes                                         96
Region                                                27
Income Group                                          27
WB-2 code                                              1
National accounts base year                           36
National accounts reference year                     209
SNA price valuation                                   44
Lending category                                      97
Other groups                                         183
System of National Accounts                           26
Alternative conversion factor                        194
PPP survey year                

In [46]:
foot_note_na

CountryCode         0
SeriesCode          0
Year                0
DESCRIPTION         0
Unnamed: 4     643638
dtype: int64

In [47]:
series_na

Series Code                               0
Topic                                     0
Indicator Name                            0
Short definition                       1509
Long definition                           0
Unit of measure                        3665
Periodicity                            3566
Base Period                            3351
Other notes                            3113
Aggregation method                     3618
Limitations and exceptions             3651
Notes from original source             3665
General comments                       3651
Source                                    0
Statistical concept and methodology    3642
Development relevance                  3662
Related source links                   3450
Other web links                        3665
Related indicators                     3665
License Type                           3665
Unnamed: 20                            3665
dtype: int64

## Description données du jeu de données (nb col. ? nb lignes ?)

In [48]:
print(f'data : {data.shape[0]} lignes, {data.shape[1]} colonnes')
print(f'country : {country.shape[0]} lignes, {country.shape[1]} colonnes')
print(f'country_series : {country_series.shape[0]} lignes, { country_series.shape[1]} colonnes')
print(f'foot_note : {foot_note.shape[0]} lignes, {foot_note.shape[1]} colonnes')
print(f'series : {series.shape[0]} lignes, {series.shape[1]} colonnes')

data : 886930 lignes, 70 colonnes
country : 241 lignes, 32 colonnes
country_series : 241 lignes, 32 colonnes
foot_note : 643638 lignes, 5 colonnes
series : 3665 lignes, 21 colonnes


## Informations pertinentes à la problématique initiale :

### Trier indicateurs pertinents

#### Enlever indicateurs non-pertinents / redondants

- age :
- income :
- 

In [61]:
# filtrer dans les descriptions des "series codes" pour sélectionner des critères selon mots clés précis
series[series["Long definition"].str.contains('15-24', na=False, case=False)]

,Series Code,Topic,Indicator Name,Short definition,Long definition,Unit of measure,Periodicity,Base Period,Other notes,Aggregation method,...,Notes from original source,General comments,Source,Statistical concept and methodology,Development relevance,Related source links,Other web links,Related indicators,License Type,Unnamed: 20
2504,SP.POP.1524.FE.UN,Population,"Population, ages 15-24, female","Population, ages 15-24, female is the total nu...","Population, ages 15-24, female is the total nu...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,UNESCO Institute for Statistics (Derived),NaN,NaN,NaN,NaN,NaN,NaN,NaN
2505,SP.POP.1524.MA.UN,Population,"Population, ages 15-24, male","Population, ages 15-24, male is the total numb...","Population, ages 15-24, male is the total numb...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,UNESCO Institute for Statistics (Derived),NaN,NaN,NaN,NaN,NaN,NaN,NaN
2506,SP.POP.1524.TO.UN,Population,"Population, ages 15-24, total","Population, ages 15-24, total is the total pop...","Population, ages 15-24, total is the total pop...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,UNESCO Institute for Statistics (Derived),NaN,NaN,NaN,NaN,NaN,NaN,NaN


###  Quels sont les pays avec un fort potentiel de clients pour nos services ?

pistes de points clés, chercher [indicateurs](https://datatopics.worldbank.org/education/indicators) avec :


- pays ayant des revenues moyens /hauts : 
        country['Income Group']
        
- gross enrolment secondary (lycée) : 
        data[data['Indicator Code'] == "SE.SEC.ENRL"]
        
- gross enrolment tertiary (uni) 
        data[data['Indicator Code'] == "SE.TER.ENRR"]

- completion



- school life expectancy primary to secondary
        data[data['Indicator Code'] == "SE.SCH.LIFE"]


- Expenditure on education as % of total government expenditure (%)
        data[data['Indicator Code'] == "UIS.XSPENDP.23.FDPUB.FNS"]
        data[data['Indicator Code'] == "UIS.XSPENDP.56.FDPUB.FNS"]


- Government expenditure on education as % of GDP (%)
        data[data['Indicator Code'] == "UIS.XGDP.56.FSGOV"]


In [ ]:
'SE.TER.ENRR', 'SE.TER.ENRR', 'SE.SCH.LIFE', 'UIS.XSPENDP.23.FDPUB.FNS', 'UIS.XSPENDP.56.FDPUB.FNS', 'UIS.XGDP.56.FSGOV'

In [62]:
country2 = country[['Country Code','Short Name','Income Group']]

In [63]:
# find distribution of different "Income Group"
country2.groupby(['Income Group']).count()

,Country Code,Short Name
Income Group,,
High income: OECD,31,31
High income: nonOECD,44,44
Low income,34,34
Lower middle income,50,50
Upper middle income,55,55


In [64]:
# select countries with category "High income: OECD"
country2.loc[country2["Income Group"] == "High income: OECD"]

,Country Code,Short Name,Income Group
11,AUS,Australia,High income: OECD
12,AUT,Austria,High income: OECD
15,BEL,Belgium,High income: OECD
33,CAN,Canada,High income: OECD
34,CHE,Switzerland,High income: OECD
36,CHL,Chile,High income: OECD
50,CZE,Czech Republic,High income: OECD
51,DEU,Germany,High income: OECD
54,DNK,Denmark,High income: OECD
65,ESP,Spain,High income: OECD


In [65]:
#
country_high = country2.loc[country2["Income Group"] == "High income: OECD"]
country_high["Country Code"]

11     AUS
12     AUT
15     BEL
33     CAN
34     CHE
36     CHL
50     CZE
51     DEU
54     DNK
65     ESP
66     EST
69     FIN
71     FRA
75     GBR
83     GRC
99     IRL
102    ISL
103    ISR
104    ITA
107    JPN
114    KOR
131    LUX
163    NLD
164    NOR
167    NZL
176    POL
179    PRT
203    SVK
204    SVN
205    SWE
227    USA
Name: Country Code, dtype: object

In [66]:
data_recent = data.iloc[: , :-18]
data_recent

,Country Name,Country Code,Indicator Name,Indicator Code,1970,1971,1972,1973,1974,1975,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Arab World,ARB,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arab World,ARB,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2.F,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arab World,ARB,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2.GPI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arab World,ARB,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2.M,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arab World,ARB,"Adjusted net enrolment rate, primary, both sex...",SE.PRM.TENR,54.822121,54.894138,56.209438,57.267109,57.991138,59.36554,...,84.011871,84.195961,85.211998,85.24514,86.101669,85.51194,85.320152,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886925,Zimbabwe,ZWE,"Youth illiterate population, 15-24 years, male...",UIS.LP.AG15T24.M,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,166679.00000,NaN,NaN,199464.000000,NaN,NaN,NaN
886926,Zimbabwe,ZWE,"Youth literacy rate, population 15-24 years, b...",SE.ADT.1524.LT.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,90.93070,NaN,NaN,90.428120,NaN,NaN,NaN
886927,Zimbabwe,ZWE,"Youth literacy rate, population 15-24 years, f...",SE.ADT.1524.LT.FE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,92.12456,NaN,NaN,93.188350,NaN,NaN,NaN
886928,Zimbabwe,ZWE,"Youth literacy rate, population 15-24 years, g...",SE.ADT.1524.LT.FM.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.02828,NaN,NaN,1.063890,NaN,NaN,NaN


In [67]:
# select rows in "data" which "Country Code" match with high OCDE list from "country_high" list previously made
data_OCED = data_recent.loc[data_recent['Country Code'].isin(country_high["Country Code"])]
data_OCED

,Country Name,Country Code,Indicator Name,Indicator Code,1970,1971,1972,1973,1974,1975,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
128275,Australia,AUS,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2,NaN,NaN,NaN,NaN,NaN,NaN,...,86.895981,87.168427,86.222816,85.811722,85.947922,86.202271,NaN,NaN,NaN,NaN
128276,Australia,AUS,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2.F,NaN,NaN,NaN,NaN,NaN,NaN,...,88.926842,89.112907,88.005913,87.499138,87.669609,87.703598,NaN,NaN,NaN,NaN
128277,Australia,AUS,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2.GPI,NaN,NaN,NaN,NaN,NaN,NaN,...,1.045980,1.043900,1.040700,1.038700,1.039490,1.034300,NaN,NaN,NaN,NaN
128278,Australia,AUS,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2.M,NaN,NaN,NaN,NaN,NaN,NaN,...,85.017868,85.365509,84.564171,84.238747,84.339340,84.795174,NaN,NaN,NaN,NaN
128279,Australia,AUS,"Adjusted net enrolment rate, primary, both sex...",SE.PRM.TENR,NaN,NaN,NaN,NaN,NaN,NaN,...,96.719856,96.839798,97.105652,97.765762,97.655258,97.450394,97.138184,97.160408,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
850275,United States,USA,"Youth illiterate population, 15-24 years, male...",UIS.LP.AG15T24.M,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
850276,United States,USA,"Youth literacy rate, population 15-24 years, b...",SE.ADT.1524.LT.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
850277,United States,USA,"Youth literacy rate, population 15-24 years, f...",SE.ADT.1524.LT.FE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
850278,United States,USA,"Youth literacy rate, population 15-24 years, g...",SE.ADT.1524.LT.FM.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
def df_search(df, column, target):
    return df[df[f"{column}"].str.contains(f'{target}', na=False, case=False)]

In [57]:
df_search(data_OCED, 'Indicator Name', 'both sexes')

,Country Name,Country Code,Indicator Name,Indicator Code,1970,1971,1972,1973,1974,1975,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
128275,Australia,AUS,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2,NaN,NaN,NaN,NaN,NaN,NaN,...,86.895981,87.168427,86.222816,85.811722,85.947922,86.202271,NaN,NaN,NaN,NaN
128279,Australia,AUS,"Adjusted net enrolment rate, primary, both sex...",SE.PRM.TENR,NaN,NaN,NaN,NaN,NaN,NaN,...,96.719856,96.839798,97.105652,97.765762,97.655258,97.450394,97.138184,97.160408,NaN,NaN
128283,Australia,AUS,"Adjusted net enrolment rate, upper secondary, ...",UIS.NERA.3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,76.045959,NaN,NaN,NaN,NaN
128287,Australia,AUS,Adjusted net intake rate to Grade 1 of primary...,UIS.NIRA.1,NaN,88.9702,NaN,91.3871,90.413681,96.962173,...,80.759453,80.921257,81.743919,NaN,NaN,83.689056,NaN,NaN,NaN,NaN
128292,Australia,AUS,"Adult illiterate population, 15+ years, both s...",UIS.LP.AG15T99,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
849951,United States,USA,"UIS: Total net attendance rate, lower secondar...",UIS.NART.2.URB,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
849955,United States,USA,Under-age enrolment ratio in primary education...,UIS.UAPP.1,NaN,NaN,NaN,NaN,NaN,NaN,...,0.934570,1.222680,0.955710,0.997420,0.893390,0.904150,0.998110,NaN,NaN,NaN
849958,United States,USA,Under-age enrolment ratio in secondary educati...,UIS.UAPP.23,NaN,NaN,NaN,NaN,NaN,NaN,...,1.130470,1.136580,1.107640,0.985800,0.909160,NaN,0.929410,NaN,NaN,NaN
850273,United States,USA,"Youth illiterate population, 15-24 years, both...",UIS.LP.AG15T24,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
# different  "UIS.XGDP.56.FSGOV|SE.SEC.ENRL|SE.TER.ENRR|SE.SCH.LIFE"

In [59]:
\b(UIS.XGDP.56.FSGOV|SE.SEC.ENRL|SE.TER.ENRR|SE.SCH.LIFE)\b

SyntaxError: unexpected character after line continuation character (2819993743.py, line 1)

In [ ]:
data_OCED[data_OCED['Indicator Code'] == "SE.SCH.LIFE"]

In [ ]:
#check missing values
# https://stackoverflow.com/questions/51070985/find-out-the-percentage-of-missing-values-in-each-column-in-the-given-dataset
percent_missing = data_OCED.isnull().sum() * 100 / len(data_OCED)
missing_value_df = pd.DataFrame({'column_name': data_OCED.columns,
                                 'percent_missing': percent_missing})
missing_value_df

#### explo

In [ ]:
#data[data['Indicator Code'].str.contains("SE.SEC.ENRL")]
data[data['Indicator Code'] == "SE.SEC.ENRL"]

###  Pour chacun de ces pays, quelle sera l’évolution de ce potentiel de clients ?

- population total
        data[data['Indicator Code'] == "SP.POP.TOTL"]
- population growth
        data[data['Indicator Code'] == "SP.POP.GROW"]
        
        
- 

### Dans quels pays l'entreprise doit-elle opérer en priorité ?

## Ordres de grandeurs des indicateurs statistiques classiques pour les différentes zones géographiques et pays du monde (moyenne/médiane/écart-type par pays et par continent ou bloc géographique)

In [ ]:
python -version